# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()


# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'hydraulic-automl-experiment'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

In [ ]:
# get the data
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
# Data Source : https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems
found = False
key = "Hydraulic Systems Data"
description_text = "Condition monitoring of hydraulic systems Data Set"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/chamsun-imoggo/udacityms-3rdproject/main/data/hydraulic_systems_training.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(ws, amlcompute_cluster_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_DS14_v2", max_nodes=10)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 20,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="STABILITY",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             enable_onnx_compatible_models=True,
                             **automl_settings
                            )


In [ ]:
# TODO: Submit your experiment
automlmodel_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(automlmodel_run).show()

In [ ]:
automlmodel_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# best run and model for automl
best_run, fitted_model = automlmodel_run.get_output()

# Print the best run
print(best_run)

# All metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='Hydraulic_BinaryClassification', model_path='outputs/model.pkl')
print(model)

In [ ]:
best_run.download_file('./outputs/model.pkl', output_file_path='./outputs')

In [ ]:
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'automl_env.yml')

In [ ]:
# set InferenceConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='./score.py',
                                    environment=best_run.get_environment())

In [ ]:
# deploy the model
from azureml.core import Model
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 1)
service = Model.deploy(workspace=ws, name="automlservice", models=[model], inference_config=inference_config, deployment_config=deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# define the test sample
import json
from azureml.data.dataset_factory import TabularDatasetFactory
test_loc = "https://raw.githubusercontent.com/chamsun-imoggo/udacityms-3rdproject/main/data/hydraulic_systems_test.csv"
test_df = TabularDatasetFactory.from_delimited_files(test_loc).to_pandas_dataframe()
test_sample = json.dumps({'data':test_df.head(5).to_dict(orient="records")})
print(test_sample)

In [ ]:
# send a request to the endpoint
import requests
headers={"Content-Type":"application/json"}

result=requests.post(service.scoring_uri, test_sample, headers=headers)
print(result)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# print service logs
print(service.get_logs())

<h2>Convert Model to ONNX</h2>

In [ ]:
best_run, onnx_model = automlmodel_run.get_output(return_onnx_model=True)

In [ ]:
# save the best ONNX model
from azureml.automl.runtime.onnx_convert import OnnxConverter
onnx_file_path = "./automl_onnx_model.onnx"
OnnxConverter.save_onnx_model(onnx_model, onnx_file_path)

In [ ]:
# delete the service 
service.delete()

In [ ]:
# delete the compute target
compute_target.delete()